<a href="https://colab.research.google.com/github/exponentialR/DeepLearningFundamentals/blob/confusion/SimpleRNN_for_number_addition_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector 
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored 

In [39]:
#Generate Data

all_chars = '0123456789+'

In [40]:
#Define number of features 
num_features = len(all_chars)
print('number of features:', num_features)
char_to_index = dict((c,i) for i, c in enumerate (all_chars))
index_to_char = dict((i, c) for i, c in enumerate (all_chars))


number of features: 11


In [41]:
#Define a function to define one single example and label
def generate_data():
  first = np.random.randint(0, 100)
  second = np.random.randint(0, 100)
  example = str(first)+ '+' + str(second)
  label = str(first+second)
  return example, label
generate_data()

('11+31', '42')

In [42]:
#Create a Model 

hidden_units = 128 
max_time_steps = 5

#Encoder 
model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
#Decoder| another RNN layer is used 
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_8 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [43]:
#Vectorize and Devectorize the Data

def Vectorize_example(example, label):
  x = np.zeros((max_time_steps, num_features))
  y = np.zeros((max_time_steps, num_features))

  diff_x = max_time_steps - len(example)
  diff_y = max_time_steps - len(label)

  for i, c in enumerate(example):
    x[i+diff_x, char_to_index[c]]=1
  for i in range (diff_x):
    x[i, char_to_index['0']] = 1
  for i, c in enumerate(label):
    y[i+diff_y, char_to_index[c]]=1
  for i in range (diff_y):
    y[i, char_to_index['0']] = 1
  return x, y 

e, l = generate_data()
print (e, l)
x, y = Vectorize_example(e, l)
print (x.shape, y.shape)

66+94 160
(5, 11) (5, 11)


In [44]:
#devectorise 

def devectorize_example(example):
  result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
  return ''.join(result)

devectorize_example(x)


'66+94'

In [45]:
devectorize_example(y)

'00160'

In [46]:
#Create Dataset 
def create_dataset(num_examples = 2000):
  x = np.zeros((num_examples, max_time_steps, num_features))
  y = np.zeros((num_examples, max_time_steps, num_features))
  for i in range (num_examples):
    e, l = generate_data()
    e_v, l_v = Vectorize_example(e, l)
    x[i] = e_v
    y[i] = l_v
  return x, y

x, y = create_dataset()
print(x.shape, y.shape)

(2000, 5, 11) (2000, 5, 11)


In [47]:
devectorize_example(x[0])

'17+58'

In [48]:
devectorize_example(y[0])

'00075'

In [50]:
#Train the Model 

l_cb = LambdaCallback(
    on_epoch_end=lambda e, l: print('{:.2f}'.format(l['val_accuracy']), end =' _ ')

)
es_cb = EarlyStopping(monitor='val_loss', patience =10)
model.fit(x, y, epochs=500, batch_size=256, validation_split=0.2,
          verbose=False, callbacks=[es_cb, l_cb])


0.60 _ 0.60 _ 0.61 _ 0.62 _ 0.62 _ 0.63 _ 0.63 _ 0.65 _ 0.65 _ 0.64 _ 0.65 _ 0.64 _ 0.65 _ 0.65 _ 0.67 _ 0.67 _ 0.67 _ 0.68 _ 0.68 _ 0.68 _ 0.70 _ 0.69 _ 0.70 _ 0.71 _ 0.71 _ 0.71 _ 0.72 _ 0.72 _ 0.72 _ 0.73 _ 0.73 _ 0.73 _ 0.74 _ 0.74 _ 0.74 _ 0.74 _ 0.74 _ 0.75 _ 0.74 _ 0.76 _ 0.76 _ 0.77 _ 0.76 _ 0.76 _ 0.77 _ 0.77 _ 0.77 _ 0.78 _ 0.79 _ 0.79 _ 0.79 _ 0.79 _ 0.80 _ 0.80 _ 0.81 _ 0.81 _ 0.81 _ 0.82 _ 0.83 _ 0.83 _ 0.84 _ 0.84 _ 0.85 _ 0.85 _ 0.85 _ 0.86 _ 0.85 _ 0.86 _ 0.87 _ 0.86 _ 0.87 _ 0.88 _ 0.88 _ 0.89 _ 0.89 _ 0.90 _ 0.89 _ 0.90 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.91 _ 0.92 _ 0.91 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.95 _ 0.93 _ 0.94 _ 0.94 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.96 _ 0.95 _ 0.96 _ 0.95 _ 0.96 _ 0.96 _ 0.96 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _

In [57]:
x_test, y_test = create_dataset(10)
preds = model.predict(x_test)

for i, pred in enumerate (preds):
  y = devectorize_example(y_test[i])
  y_hat = devectorize_example(pred)
  col ='green'
  if y!=y_hat:
    col ='red'
  out = 'Input: '+devectorize_example(x_test[i])+' Out: '+y+' Pred: '+y_hat
  print(colored(out, col))


Input: 74+64 Out: 00138 Pred: 00138
Input: 12+72 Out: 00084 Pred: 00084
Input: 78+80 Out: 00158 Pred: 00158
Input: 84+72 Out: 00156 Pred: 00156
Input: 45+13 Out: 00058 Pred: 00058
Input: 44+26 Out: 00070 Pred: 00070
Input: 35+63 Out: 00098 Pred: 00098
Input: 87+21 Out: 00108 Pred: 00108
Input: 98+60 Out: 00158 Pred: 00158
Input: 22+98 Out: 00120 Pred: 00120
